In [4]:
import pandas as pd

df = pd.read_excel('5.5_viz_dataset.xlsx')
df = df.rename(columns={'genre_one': 'genre'})

# 매출액 열을 숫자로 변환
df['매출액'] = pd.to_numeric(df['매출액'].str.replace(',', ''), errors='coerce')

# 매출액 열의 결측치가 있는 행 제거
df = df.dropna(subset=['매출액'])


import plotly.express as px

# 개봉일을 datetime 형식으로 변환
df['개봉일'] = pd.to_datetime(df['개봉일'])

# 개봉일에서 연도 정보 추출
df['연도'] = df['개봉일'].dt.year

# 배급사에 따른 연도별 매출액 합계 계산
sales_by_company_year = df.groupby(['배급사', '연도']).sum()['매출액'].reset_index()

# 배급사에 따른 연도별 관객수 합계 계산
audience_by_company_year = df.groupby(['배급사', '연도']).sum()['관객수'].reset_index()

# 각 연도별로 상위 10개 배급사 선택
top_sales_companies_by_year = sales_by_company_year.groupby('연도').apply(lambda x: x.nlargest(10, '매출액')).reset_index(drop=True)
top_audience_companies_by_year = audience_by_company_year.groupby('연도').apply(lambda x: x.nlargest(10, '관객수')).reset_index(drop=True)

# 매출액 시각화 (상위 10개 배급사)
fig_sales_top10 = px.bar(top_sales_companies_by_year, x='연도', y='매출액', color='배급사', title='연도별 상위 10개 배급사의 매출액',
                         color_discrete_sequence=px.colors.qualitative.Alphabet)
# 매출액의 y축을 회계 형식으로 설정
fig_sales_top10.update_layout(yaxis_tickformat=',.0f')
fig_sales_top10.show()
fig_sales_top10.write_html('hj_sales.html')

# 관객수 시각화 (상위 10개 배급사)
fig_audience_top10 = px.bar(top_audience_companies_by_year, x='연도', y='관객수', color='배급사', title='연도별 상위 10개 배급사의 관객수',
                            color_discrete_sequence=px.colors.qualitative.Alphabet)
# 매출액의 y축을 회계 형식으로 설정
fig_audience_top10.update_layout(yaxis_tickformat=',.0f')
fig_audience_top10.show()
fig_audience_top10.write_html('hj_audience.html')


/var/folders/19/klzzjsv901qcj2xxsvpj2jqw0000gn/T/ipykernel_71265/1057743479.py:22: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

/var/folders/19/klzzjsv901qcj2xxsvpj2jqw0000gn/T/ipykernel_71265/1057743479.py:25: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [ ]:
# 배급사별 가장 많은 매출을 올린 영화 찾기
top_movies = df.loc[df.groupby('배급사')['매출액'].idxmax()]

# 매출액을 기준으로 내림차순으로 정렬하고 상위 10개만 선택
top_movies = top_movies.sort_values(by='매출액', ascending=False).head(10)

# 시각화
fig = px.bar(top_movies, x='배급사', y='매출액', color='genre_one', title='배급사별 가장 많은 매출을 올린 영화의 장르 (상위 10개)')

# 매출액의 y축을 회계 형식으로 설정
fig.update_layout(yaxis_tickformat=',.0f')
fig.show()
fig.write_html('yj_money.html')

## 장르별 가장 많은 매출을 기록한 영화 찾기

In [ ]:
# 장르별 가장 많은 매출을 기록한 영화 찾기
top_movies_by_genre = df.loc[df.groupby('genre_one')['매출액'].idxmax()]

# 매출액을 기준으로 내림차순으로 정렬하고 상위 10개만 선택
top_movies_by_genre = top_movies_by_genre.sort_values(by='매출액', ascending=False).head(10)

# 시각화
fig = px.bar(top_movies_by_genre, x='genre_one', y='매출액', color='배급사', title='장르별 가장 많은 매출을 기록한 배급사 (상위 10개)')

# 매출액의 y축을 회계 형식으로 설정
fig.update_layout(yaxis_tickformat=',.0f')

fig.show()
fig.write_html('yj_genre.html')